In [1]:
%matplotlib inline

from IPython.core.display import *
import operator

# Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go
from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol)
is a node and there are edges to the north, south, east and west (except at the edges).

## The World

The world is represented as a List of Lists of symbols that indicate the terrain:

In [2]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '^', '^', '^', 'x', 'x', '^', '^'], 
  ['.', '.', '.', '.', '^', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '^', '^', 'x', 'x', '^', '.'], 
  ['.', '.', '.', '^', '^', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '^', 'x', 'x', 'x', '^', '.'], 
  ['.', '^', '^', '^', 'x', 'x', '^', '^', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '^', 'x', '^', '.', '.'], 
  ['.', '^', '^', 'x', 'x', '^', '^', '.', '.', '.', '.', '^', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '^', '.', '.', '.'], 
  ['.', '.', '^', '^', '^', '^', '^', '.', '.', '.', '.', '.', '.', '^', 'x', 'x', 'x', '~', '~', '~', '.', '.', '^', '^', '^', '.', '.'], 
  ['.', '.', '.', '^', '^', '^', '.', '.', '.', '.', '.', '.', '^', '^', 'x', 'x', '.', '~', '~', '.', '.', '^', '^', '^', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '^', '^', '^', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '^', '^', '^', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '^', '^', 'x', 'x', 'x', '^', '.', '.', '.', '.', '.', '^', 'x', 'x', 'x', '^', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '^', 'x', 'x', '^', '.', '.', '.', '.', '~', '~', '.', '.', '^', 'x', 'x', '^', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '^', '^', 'x', 'x', '^', '.', '~', '~', '~', '~', '.', '.', '.', '^', 'x', '^', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '^', '*', '*', '^', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '^', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '^', '^', '^', '^', '.', '~', '~', '~', '.', '.', '^', 'x', '^', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '^', '^', '.', '~', '.', '^', 'x', 'x', '^', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '^', '^', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '^', '^', '^', '^', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '^', '^', '^', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '^', '^', '^', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '^', '^', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '^', '^', '.', '.', '^', '^', '^', '^', '^', '.', '.', '.', '.', '.', '^', '^', 'x', 'x', '^', '^', '.', '~', '~', '~', '~', '~'], 
  ['^', 'x', '^', '^', '^', '^', '.', '.', '.', '.', '.', 'x', 'x', 'x', '^', '^', 'x', 'x', '.', 'x', 'x', '^', '^', '~', '~', '~', '~'], 
  ['^', 'x', 'x', 'x', '^', '.', '.', '.', '.', '.', '^', '^', 'x', 'x', 'x', 'x', '^', '^', '^', '^', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['^', '^', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '^', '^', '^', '^', '^', '.', '.', '.', '.', '^', '^', '^', '.', '.', '.']]

**Important Note**

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7).

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics, what do you expect the policy would be?

In [3]:
test_world = [
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

In [4]:
test_world2 = [
  ['.', 'x', '*', '*', '*', '*', '*'],
  ['.', 'x', '*', 'x', '.', '.', '.'],
  ['.', 'x', '*', 'x', '.', 'x', '.'],
  ['.', '*', '.', 'x', '.', 'x', '.'],
  ['.', '.', '*', 'x', '.', 'x', '.'],
  ['.', '*', '*', 'x', '.', '.', '.'],
  ['.', '*', '*', '*', '*', 'x', '.'],
]

Remember that a state space search problem is formalized as States, Actions, Transitions and Costs. The map itself represents states and transitions. Costs is fairly easy:

## Costs

We can encode the costs described above in a Dictionary:

In [5]:
COSTS = { '.': 1, '*': 3, '^': 5, '~': 7}

## Movement Model and Actions

We can handle actions by both specifying a movement model and then checking for valid moves in our `successor` function.

The movement model is a list of offsets for neighbors. When given the current state (x, y), it tells you the adjacent states. Note that not all such adjacent states are legal moves, for example, if (0,0) is the current state then (0, 0) + (0, -1) = (0, -1) which is off the map. Or if the current state is (5,5) and (5,6) is a mountain, then (0,1) would not be a legal move. You will need to check if a move is legal or not.

In [6]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## A* Search

Implement your code for A* search below. It should take the following form:

```
    def a_star_search( world, costs, start, goal, moves):
```

where `world` is a list of lists of terrain symbols, `costs` is a dictionary, `start` is a tuple (x, y) representing the starting coordinate, `goal` is a tuple (x,y) representing the goal coordinate, and `moves` is a list of tuples that represent (x,y) offsets.

The function returns a list of tuples that represent the coordinates on the path from start to goal:

[(0,0), (0,1), ...., (26, 26)]

You should also write a function that prints out the world and the path over it as "ASCII" art:

```
    def show_path( world, path):
```

It has no return value. Remember how (x, y) is world[ y][ x].

From here onward, you are to code up your implementation, using Markdown and Code blocks to write helper functions and comments for the `a_star_search` function. Like so:

-----

#Function get_position() is used to add two tuple move (x1,y1) and (x2,y2) together. The function returns new tuple move (x,y) with x=x1+x2 and y=y1+y2.

In [7]:
def get_position(current, move):
    return tuple(map(operator.add, current, move))

#Function is_legal_move() is used to check whether the new move is legal. The function returns False if the move is not in the world or the move is on mountain 'x' or the move is not plain, forrest, hill, and swamp; otherwise, it returns True.

In [8]:
def is_legal_move(world, current):
    if current[0] < 0 or current[0] >= len(world[0]):
        return False
    if current[1] < 0 or current[1] >= len(world[0]):
        return False
    if world[current[1]][current[0]] == 'x':
        return False
    tokens = ['.', '*', '^', '~']
    if world[current[1]][current[0]] not in tokens:
        return False
    return True

#Function get_heuristic() is the heuristic function h used to calculate the heuristic distance from current to goal position. The heuristic function uses Manhattan distance h = d*(abs(x1-x2)+abs(y1-y2)). d is the minimum cost for moving to other terrains (For example, d of plain is 1).  

In [9]:
def get_heuristic(world, costs, current, goal):
    token = world[current[1]][current[0]]
    d = costs[token]
    dx = abs(current[0] - goal[0])
    dy = abs(current[1] - goal[1])
    return d*(dx+dy)

#Function get_cost() is the cost function g used to calculate the cost distance from start to current position. It returns the cost from start to current position.

In [10]:
def get_cost(world, costs, current, prev_cost):
    token = world[current[1]][current[0]]
    return prev_cost + costs[token]

#Function is_empty() is used to check whether the list is empty. It is particularly used to check whether the Frontier list is empty or not. It returns True if the list is empty; otherwise, it returns False. 

In [11]:
def is_empty(l):
    if l:
        return False
    else:
        return True

#Function is_in_frontier() is used to check whether the current move is in the frontier list or not. It returns -1 if the move is not in the frontier list; otherwise, it returns index of the move in the frontier list. 

In [12]:
def is_in_frontier(frontier, current):
    for i in range(0,len(frontier)):
        if current == frontier[i][3]:
            return i
    return -1

#Function build_path() is used to build the shortest path list generated from the A star search. The parameter path is a dictionary and it is like a bookkeeping that keep tracking moves of the search. path[goal] will return the previous move from goal position, and the function keep loop through the dictionary path until it finds the start to form the path list. 

In [13]:
def build_path(path, start, goal):
    ret_path = [goal]
    current = goal
    while path[current] != start:
        current = path[current]
        ret_path.append(current)
    ret_path.append(start)
    ret_path.reverse()
    return ret_path

###Function a_start_search is used to find the shortest path from start to goal in world. Possible moves are defined in moves. Cost of each move is defined in costs. Since the world is a square grid that allows 4 directions of movement (up, down, left, right), the heuristic function used for the below algorithm is "Manhanttan Distance" h = d*(abs(x1-x2)+abs(y1-y2)), and d is the minimum cost for moving to other terrains (For example, d of plain is 1). The function returns a list of moves of the shortest path from start to goal, and it returns empty list if it can't find the path from start to goal.

###Pseudo code
Explorer = empty list

Frontier = empty list

Path = empty dictionary

f = g of start + h of start to goal

push tuple (f,g,h,start) to Frontier

while Frontier is not empty

    current = the tuple in Frontier having lowest f value
    if current is goal
        return the shortest path
    push current to Explorer
    for each move in moves
        get the next move
        if the next move is not valid, skip
        if the next move is already in Explorer, skip
        Calculate f = g + h of the next move
        push next move to Frontier if it not yet in Frontier
        Update f value of next move in Frontier if it is already in Frontier
        sort Frontier based on f value
        Path[next_move] = current
        
return failure []

In [14]:
def a_star_search( world, costs, start, goal, moves):
    if is_legal_move(world, start) == False:
        return []
    if is_legal_move(world, goal) == False:
        return []
    Explorer = []
    Frontier = []
    Path = {}
    g = get_cost(world, costs, start, 0)
    h = get_heuristic(world, costs, start, goal)
    f = g + h
    Frontier.append((f,g,h,start))
    while not is_empty(Frontier):
        current = Frontier.pop(0)
        if current[3] == goal:
            return build_path(Path, start, goal)
        Explorer.append(current[3])
        for move in moves:
            next_move = get_position(current[3], move)
            if is_legal_move(world, next_move) == False:
                continue
            if next_move in Explorer:
                continue
            i = is_in_frontier(Frontier, next_move)
            g = get_cost(world, costs, next_move, current[1])
            h = get_heuristic(world, costs, next_move, goal)
            f = g + h
            if i == -1:
                Frontier.append((f,g,h,next_move))
            elif (f < Frontier[i][0]):
                Frontier[i] = (f,g,h,next_move)
            Frontier.sort()
            Path[next_move] = current[3]
    return []

#Function print_path() is used to print the shortest path from start to goal using A star search. It prints out both (x,y) moves and moves marked in "()" in grid square world.

In [15]:
def print_path( world, path):
    if path:
        str_path = "The path (x,y):\n"
        for p in path:
            str_path += "->"+str(p)
        print str_path
        str_map = "The path map:\n"
        for x in range(0, len(world[0])):
            for h in range(0, len(world[0])):
                str_map += '----'
            str_map += '-\n'
            for y in range(0, len(world[0])):
                if (y,x) in path:
                    str_map += '|' + '(' + world[x][y] + ')'
                else:
                    str_map += '| ' + world[x][y] + ' '
            str_map += '|\n'
        for h in range(0, len(world[0])):
            str_map += '----'
        str_map += '-\n'
        print str_map
    else:
        print "Couldn't find the path"

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

In [16]:
test_path = a_star_search( test_world, COSTS, (0, 0), (6, 6), MOVES)
print_path( test_world, test_path)

The path (x,y):
->(0, 0)->(0, 1)->(0, 2)->(0, 3)->(1, 3)->(2, 3)->(3, 3)->(4, 3)->(5, 3)->(6, 3)->(6, 4)->(6, 5)->(6, 6)
The path map:
-----------------------------
|(.)| * | * | * | * | * | * |
-----------------------------
|(.)| * | * | * | * | * | * |
-----------------------------
|(.)| * | * | * | * | * | * |
-----------------------------
|(.)|(.)|(.)|(.)|(.)|(.)|(.)|
-----------------------------
| * | * | * | * | * | * |(.)|
-----------------------------
| * | * | * | * | * | * |(.)|
-----------------------------
| * | * | * | * | * | * |(.)|
-----------------------------



In [17]:
test_path2 = a_star_search( test_world2, COSTS, (0, 0), (6, 6), MOVES)
print_path( test_world2, test_path2)

The path (x,y):
->(0, 0)->(0, 1)->(0, 2)->(0, 3)->(0, 4)->(0, 5)->(0, 6)->(1, 6)->(2, 6)->(3, 6)->(4, 6)->(4, 5)->(5, 5)->(6, 5)->(6, 6)
The path map:
-----------------------------
|(.)| x | * | * | * | * | * |
-----------------------------
|(.)| x | * | x | . | . | . |
-----------------------------
|(.)| x | * | x | . | x | . |
-----------------------------
|(.)| * | . | x | . | x | . |
-----------------------------
|(.)| . | * | x | . | x | . |
-----------------------------
|(.)| * | * | x |(.)|(.)|(.)|
-----------------------------
|(.)|(*)|(*)|(*)|(*)| x |(.)|
-----------------------------



In [18]:
full_path = a_star_search( full_world, COSTS, (0, 0), (26, 26), MOVES)
print_path( full_world, full_path)

The path (x,y):
->(0, 0)->(1, 0)->(2, 0)->(3, 0)->(4, 0)->(4, 1)->(5, 1)->(6, 1)->(7, 1)->(8, 1)->(8, 2)->(8, 3)->(8, 4)->(8, 5)->(8, 6)->(7, 6)->(7, 7)->(7, 8)->(7, 9)->(7, 10)->(7, 11)->(6, 11)->(6, 12)->(6, 13)->(6, 14)->(7, 14)->(8, 14)->(9, 14)->(10, 14)->(10, 15)->(11, 15)->(11, 16)->(12, 16)->(12, 17)->(12, 18)->(12, 19)->(13, 19)->(14, 19)->(15, 19)->(16, 19)->(17, 19)->(18, 19)->(19, 19)->(19, 20)->(19, 21)->(19, 22)->(20, 22)->(20, 23)->(21, 23)->(21, 24)->(22, 24)->(23, 24)->(23, 25)->(23, 26)->(24, 26)->(25, 26)->(26, 26)
The path map:
-------------------------------------------------------------------------------------------------------------
|(.)|(.)|(.)|(.)|(.)| * | * | * | * | * | * | * | * | * | * | . | . | . | . | . | . | . | . | . | . | . | . |
-------------------------------------------------------------------------------------------------------------
| . | . | . | . |(.)|(.)|(.)|(*)|(*)| * | * | * | * | * | * | * | . | . | x | x | x | x | x | x | x | . | . |
------